In [114]:
import pandas as pd
import os
import re
import csv

In [115]:
raw_path = 'file.csv'
clean_path = "./../../data/clean/cleaned_news_unlabeled.csv"
splitted_path = "./../../data/clean/splitted/"

data=pd.read_csv("./../../data/news_unlabeled.csv")

In [116]:
data['sentence'] = data['sentence'].str.replace(r'https?://[^\s]+', lambda x: x.group(0).replace('.', 'DOT'), regex=True)
data_cleaned = data['sentence'].str.replace(r'[^\w\s\+\/\.\-\:]', '', regex=True)
data_cleaned = data_cleaned.str.replace(r'DOT', '.', regex=False)

data_cleaned = data_cleaned.str.replace(r'(?i)\b(jakarta\s+kompas\.com|kompas\.com\s*-|kompas\.com\s+|\bkompas\.com\b)\b', '', regex=True)

data_cleaned = data_cleaned.str.replace(r'^[\s\-/]+', '', regex=True)

data_cleaned = data_cleaned.str.replace(r'\bsumber\s+(https?:[^\s]+|www\.[^\s]+)|https?:[^\s]+|www\.[^\s]+', '', regex=True)

In [117]:
data_cleaned = data_cleaned.fillna('')
data_cleaned = data_cleaned[data_cleaned.str.strip() != '']

In [118]:
os.makedirs(os.path.dirname(clean_path), exist_ok=True)
data_cleaned.to_csv(clean_path, index=False)
print(f"File berhasil dibersihkan dan disimpan di: {clean_path}")

File berhasil dibersihkan dan disimpan di: ./../../data/clean/cleaned_news_unlabeled.csv


## Split CSV for labelling

In [119]:
split_size = len(data_cleaned) // 4
names = ['kenzie', 'naufal', 'hafiz', 'satrio']
for i, name in enumerate(names):
    start_idx = i * split_size
    if i == len(names) - 1:  
        end_idx = len(data_cleaned)
    else:
        end_idx = (i + 1) * split_size
    
    split_data = data_cleaned.iloc[start_idx:end_idx]
    os.makedirs(os.path.dirname(splitted_path), exist_ok=True)
    split_file_path = os.path.join(os.path.dirname(splitted_path), f"unlabeled_{name}.csv")
    split_data.to_csv(split_file_path, index=False)
    print(f"File untuk {name} berhasil disimpan di: {split_file_path}")

File untuk kenzie berhasil disimpan di: ./../../data/clean/splitted\unlabeled_kenzie.csv
File untuk naufal berhasil disimpan di: ./../../data/clean/splitted\unlabeled_naufal.csv
File untuk hafiz berhasil disimpan di: ./../../data/clean/splitted\unlabeled_hafiz.csv
File untuk satrio berhasil disimpan di: ./../../data/clean/splitted\unlabeled_satrio.csv
